In [193]:
import os
import pandas as pd
import zstandard
import tiktoken
import io

In [195]:
# encoder for gpt4, gpt3.5 and text-embedding-ada-002
enc = tiktoken.get_encoding("cl100k_base")

In [208]:
filename = '00'

In [211]:
def process_json_line(obj):
    # get the dataset
    pile_set_name = obj["meta"]["pile_set_name"]
    tokens = len(enc.encode(obj["text"], disallowed_special=()))
    return { "pile_set_name": pile_set_name, "tokens": tokens }

In [212]:
# uncompress the zstd file and read the first few lines of the test.jsonl file
with open(os.path.join('..', 'data', 'thepile', filename + '.jsonl.zst'), 'rb') as fh:
    dctx = zstandard.ZstdDecompressor()
    # Create a streaming decompressor
    decompressor = dctx.stream_reader(fh)
    # Create a text I/O object that will be used to read the decompressed text
    text_stream = io.TextIOWrapper(decompressor, encoding='utf-8')

    data = []
    i = 0
    for line in text_stream:
        json_obj = json.loads(line)
        data.append(process_json_line(json_obj))
        i += 1
        if i % 10000 == 0:
            print(i)

10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
1390

In [213]:
df = pd.DataFrame(data)

In [214]:
#check the amount of memory the dataframe is using
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7021438 entries, 0 to 7021437
Data columns (total 2 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   pile_set_name  object
 1   tokens         int64 
dtypes: int64(1), object(1)
memory usage: 510.4 MB


In [215]:
df.shape

(7021438, 2)

In [216]:
df.head()

,pile_set_name,tokens
0,Pile-CC,2952
1,Github,1003
2,OpenWebText2,250
3,Pile-CC,80
4,Pile-CC,360


In [217]:
df['pile_set_name'].value_counts()

Pile-CC              1748721
OpenWebText2         1079800
StackExchange         984049
PubMed Abstracts      977059
Github                602131
Wikipedia (en)        563984
USPTO Backgrounds     369942
PubMed Central        189846
FreeLaw               169085
ArXiv                  79046
DM Mathematics         63503
NIH ExPorter           59552
HackerNews             52361
Enron Emails           30669
OpenSubtitles          21118
YoutubeSubtitles       11038
Books3                  9364
EuroParl                4292
Gutenberg (PG-19)       2198
PhilPapers              2192
BookCorpus2              865
Ubuntu IRC               623
Name: pile_set_name, dtype: int64

In [220]:
df.to_parquet(os.path.join('..', 'data', 'thepile', filename + '-counts-noindex.parquet'), index=False, compression='gzip')

: 